In [1]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
from matplotlib.patches import Circle
%matplotlib inline
import numpy as np
from math import atan2
import matplotlib.pyplot as plt
import importlib

import spline
importlib.reload(spline)
from spline import minsnap

import sense
importlib.reload(sense)
from sense import Sense

import robot
importlib.reload(robot)
from robot import Robot

import target
importlib.reload(target)
from target import Target

/shared-libs/python3.8/py/lib/python3.8/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.17.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'\n#######################################\n# ZMB Goals\n\n- Get the ellipse working\n- Ensure motion is working properly\n- Ensure trajectory for robot is following target\n- Make sure robot continues to follow target as long as it is moving\n\n\n'

In [3]:

######################################################################################################################################################################
#####
#####              ANIMATION
#####
######################################################################################################################################################################



# This is the animation code from HW6
from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
from matplotlib.patches import Circle, Ellipse, Rectangle
size = 5
def x_d(t):
  return np.zeros(size)

def create_animation(xTarget ,xRobot, tf, Arena, targetUncertainty=[], robotSize=0.1,targetSize=0.1,tailLength=10,n_frames = 60,uncertaintyFilterFlag=0):
  # Sample desired trajectory
  n_samples = 1000
  t_samples = np.linspace(0.0, tf, n_samples)
  x_des = np.zeros((n_samples, size))
  for i in range(t_samples.shape[0]):
    x_des[i] = x_d(t_samples[i])

  x_des = xTarget
  x = xRobot

  from matplotlib import rc
  rc('animation', html='jshtml')

  fig = plt.figure(figsize=(16,12))
  ax = plt.axes()

  x_max = max(np.max(x_des[:, 0]), np.max(x[:, 0]))
  x_min = min(np.min(x_des[:, 0]), np.min(x[:, 0]))
  y_max = max(np.max(x_des[:, 1]), np.max(x[:, 1]))
  y_min = min(np.min(x_des[:, 1]), np.min(x[:, 1]))
  
  frame_idx = [round(x) for x in np.linspace(0, xRobot.shape[0]-1, n_frames).tolist()]
  x_Robot = np.zeros((n_frames, size))
  x_Target = np.zeros((n_frames, size))
  for i in range(n_frames):
    x_Robot[i, :] = xRobot[frame_idx[i], :]
    x_Target[i, :] = xTarget[frame_idx[i], :]
    targetUncertainty[i, :] = targetUncertainty[frame_idx[i], :]

  a = 0.25
#   y = x_anim[:, 0]
#   z = x_anim[:, 1]
#   theta = x_anim[:, 2]

  x_padding = 0.25 * (x_max - x_min)
  y_padding = 0.25 * (y_max - y_min)

  def frame(i):
    ax.clear()

    # Need to set up the frame, which will consist of the arena, target dot, target ellipse, robot
    # Handle the arena
    # arenaPatch = Circle((0,0),radius=Arena.arenaSize,edgecolor='r',facecolor='None')
    arenaPatch = Rectangle((arena.arenaSize[0,0],arena.arenaSize[1,0]),arena.arenaSize[0,1]-arena.arenaSize[0,0],arena.arenaSize[1,1]-arena.arenaSize[1,0],edgecolor='r',facecolor='None')
    ax.add_patch(arenaPatch)

    # Handle the target
    if targetSize>0:
        targetPatch = Circle(x_Target[i,0:2],radius=targetSize,edgecolor='b',facecolor='b',label='Target')
        ax.add_patch(targetPatch)
    else:
        ax.plot(x_Target[i,0],x_Target[i,1],'b*',label='Target')

    # Create an ellipse to represent the uncertainty area

    # Uncertainty array has the following structure:
    # Index 0,1: Mean
    # Index 2,3: w1
    # Index 4,5: w2
    # Index 6,7: w3
    # Index 8,9: w4
    # Index 10: theta
    # Index 11,12: wStar

    if targetUncertainty.shape[0]>0:
        ellipseCenter = (targetUncertainty[i,0],targetUncertainty[i,1])
        # print(f'Ellipse Center: {ellipseCenter}')
        if uncertaintyFilterFlag==1:
            ellipseWidth = np.sqrt((targetUncertainty[i,2]-targetUncertainty[i,4])**2+(targetUncertainty[i,3]-targetUncertainty[i,5])**2)
            ellipseLength = np.sqrt((targetUncertainty[i,6]-targetUncertainty[i,8])**2+(targetUncertainty[i,7]-targetUncertainty[i,9])**2)
            ellipseAngle = targetUncertainty[i,10]
        else:
            ellipseWidth = np.sqrt(targetUncertainty[i,2]**2+targetUncertainty[i,3]**2)*2
            ellipseLength = np.sqrt(targetUncertainty[i,4]**2+targetUncertainty[i,5]**2)*2
            ellipseAngle = targetUncertainty[i,6]
        uncertaintyPatch = Ellipse(ellipseCenter,ellipseWidth,ellipseLength,ellipseAngle,edgecolor='b',facecolor='None',ls='--')
        ax.add_patch(uncertaintyPatch)

        # Will also add a red cross to signify the estimated position:
        ax.plot(ellipseCenter[0],ellipseCenter[1],'rx',label='Estimated Target Position')
        ax.plot(targetUncertainty[i,11],targetUncertainty[i,12],'b*',label='wStar')


    # Handle the robot
    if robotSize>0:
        robotPatch = Circle(x_Robot[i,0:2],radius=robotSize,edgecolor='g',facecolor='g',label='Robot')
        ax.add_patch(robotPatch)
    else:
        ax.plot(x_Robot[i,0],x_Robot[i,1],'go',label='Robot')



    prevPosIndex = np.clip(tailLength,0,i)
    # ax.plot(x_des[0, 0], x_des[0, 1], 'b*', label='desired position')
    ax.plot(x_Target[i-prevPosIndex:i+1, 0], x_Target[i-prevPosIndex:i+1, 1], 'b--', label='target trajectory')
    ax.plot(x_Robot[i-prevPosIndex:i+1, 0], x_Robot[i-prevPosIndex:i+1, 1], 'g--', label='robot trajectory')
    ax.text(Arena.arenaSize[0,1]-15,Arena.arenaSize[1,1]+4,'Simulation Time: '+str(x_Robot[i,-1])+' s')
    # plot=ax.scatter(x_anim[i, 0], x_anim[i, 1], c='r', label='quadrotor position')
    # plot = ax.plot([y[i] + a*cos(theta[i]), y[i] - a*cos(theta[i])],
    #                [z[i] + a*sin(theta[i]), z[i] - a*sin(theta[i])] , 'g','LineWidtheeta',3)
    
    # if(np.abs((x_max - x_min) - (y_max - y_min)) < 5):
    #   ax.set_xlim(x_min - x_padding, x_max + x_padding)
    #   ax.set_ylim(y_min - y_padding, y_max + y_padding)
    ax.set_xlim(Arena.arenaSize[0,0]*1.1,Arena.arenaSize[0,1]*1.1)
    ax.set_ylim(Arena.arenaSize[1,0]*1.1,Arena.arenaSize[1,1]*1.1)
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_aspect('equal')
    ax.legend(loc='upper left')

    return ax

  return animation.FuncAnimation(fig, frame, frames=n_frames, blit=False, repeat=False), fig
  


In [5]:
class Arena(object):
    def __init__(self,arenaSize):
        self.arenaSize = arenaSize

In [6]:
# Runs the simulation
from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
from matplotlib.patches import Circle
%matplotlib inline
import numpy as np
from math import atan2
import matplotlib.pyplot as plt
import importlib
import random

import spline
importlib.reload(spline)
from spline import minsnap

import sense
importlib.reload(sense)
from sense import Sense

import robot
importlib.reload(robot)
from robot import Robot

import target
importlib.reload(target)
from target import Target

createAnimationFlag=True


# Parameters
arenaSize = np.array([[0,160],[0,90]])   # meters
targetSize = 0.5 # meters (entering 0 means the target is a point mass)
robotSize = 0.5 # meters (entering 0 means the robot is a point mass)
maxNoise = 0.5 # meters


alpha = .0001
vMax = 5.
aMax = 10.
jMax = 20.
bearingRange = 30  # Not Used
bearingRate = 4  # Not Used
sampFreq = 100.
simulationTime = 60.
predictionSteps = 10 # MPC Window Size
points = np.array([[40.,50.],[35.,60.],[25.,40.],[40.,50.]]) 
# x = np.arange(100, 60, -.5)
# y = 5*np.sin(x)+40
# points = np.vstack((x,y)).T

stationaryTime = 300  # Time to let particle filter saturate


# Lets create random beginning and end points for target
randX1 = np.round(random.uniform(arenaSize[0,0],arenaSize[0,1]))
randY1 = np.round(random.uniform(arenaSize[1,0],arenaSize[1,1]))

randX2 = np.round(random.uniform(arenaSize[0,0],arenaSize[0,1]))
randY2 = np.round(random.uniform(arenaSize[1,0],arenaSize[1,1]))

# icTarget = np.array([randX1,randY1,0.,0.,0.,0.],dtype=float).T
# fcTarget = np.array([randX2,randY2,0.,0.,0.,0.],dtype=float).T

icTarget = np.array([100., 40.,0.,0.,0.,0.],dtype=float).T
fcTarget = np.array([40., 50.,0.,0.,0.,0.],dtype=float).T

# Lets create a random starting point for the robot
randX3 = np.round(random.uniform(arenaSize[0,0],arenaSize[0,1]))
randY3 = np.round(random.uniform(arenaSize[1,0],arenaSize[1,1]))

# icRobot = np.array([randX3,randY3,0.,0.,0.,0.],dtype=float).T
icRobot = np.array([20.,20.,0.,0.,0.,0.],dtype=float).T

# Class Initialization
arena = Arena(arenaSize)
target = Target(arenaSize,[vMax,aMax,jMax,bearingRange,bearingRate,sampFreq])
robot = Robot(arenaSize,[vMax,aMax,jMax,bearingRange,bearingRate,sampFreq],icRobot,icTarget,order=6,predictionSteps=predictionSteps)
sense = Sense(icTarget[0:2], arenaSize, maxNoise)


# Main Simulation Loop
target.trajectorySelector(0,icTarget,fcTarget,simulationTime)
'''

Robot functions:
- setRobot -> Get robot position
- getU -> Output robot control input
- getSense -> Get target Position

Sense functions:
- setTarget -> Get target position with noise already injected
- PF -> Particle filter
- RegionApprx -> Region approximation

'''

count = 0
dist2Target = 1000
dist2WSTAR = 1000
distThresh = 0.01



maxCount = simulationTime*sampFreq
finalTargetTrajectory = []
finalRobotTrajectory = []
targetDist = []
dualTraj = []
uncertaintyArray = []

updateCounter = 0
wStar=np.zeros((2,))

while (count<maxCount and (dist2Target>=distThresh or dist2WSTAR>=distThresh)):

    targetTraj = target.trajectory[count]

    randTarget = np.array((targetTraj[0]+random.uniform(-maxNoise,maxNoise),targetTraj[1]+random.uniform(-maxNoise,maxNoise))).reshape((-1))
    # print(randTarget)
    # w1, w2, w3, w4, theta, particles, weights, mean, covariance_matrix = sense.particle_filter(targetTraj[0:2])
    w1, w2, w3, w4, theta, particles, weights, mean, covariance_matrix = sense.particle_filter(randTarget)
    
    # Theta comes out as radians, ellipse needs it as degrees
    theta = (theta/np.pi) * 180
    wStar = robot.omega_star(wStar, alpha, particles, weights, mean, w1, w2, w3, w4)
    uncertaintyArray.append(np.hstack((mean,w1,w2,w3,w4,theta,wStar)))
    finalTargetTrajectory.append(targetTraj)

    robot.updateTargetPos(mean,wStar)

    if count>stationaryTime:
        if count%predictionSteps==0: 
            robot.createTrajectory(scaleFlag=False,sophisticatedTimeScalingFlag=True,mpcFlag=False)
    
    try:
        robot.updateCurrentState()
    except IndexError:
        dist2Target = np.sqrt((robot.currentState[0]-targetTraj[0])**2 + (robot.currentState[1]-targetTraj[1])**2)
        dist2WSTAR = np.sqrt((robot.currentState[0]-wStar[0])**2 + (robot.currentState[1]-wStar[1])**2)
        distWSTAR2Target = np.sqrt((targetTraj[0]-wStar[0])**2 + (targetTraj[1]-wStar[1])**2)
        targetDist.append([dist2Target,dist2WSTAR,distWSTAR2Target])
        break
        

    dist2Target = np.sqrt((robot.currentState[0]-targetTraj[0])**2 + (robot.currentState[1]-targetTraj[1])**2)
    dist2WSTAR = np.sqrt((robot.currentState[0]-wStar[0])**2 + (robot.currentState[1]-wStar[1])**2)
    distWSTAR2Target = np.sqrt((targetTraj[0]-wStar[0])**2 + (targetTraj[1]-wStar[1])**2)
    targetDist.append([dist2Target,dist2WSTAR,distWSTAR2Target])     
    count += 1
    
uncertaintyArray = np.array(uncertaintyArray)
tf = count*sampFreq

targetUncertainty = np.ones((target.trajectory.shape[0],7))*1
targetUncertainty[:,0] = fcTarget[0]
targetUncertainty[:,1] = fcTarget[1]
targetUncertainty[:,2] = 2
targetUncertainty[:,3] = 2
targetUncertainty[:,4] = -1
targetUncertainty[:,-1] = 45

################################################################################
#####  ANIMATION  ##############################################################
################################################################################

finalTargetTrajectory = np.array(finalTargetTrajectory)
robotTime = np.arange(0,robot.finalTrajectory.shape[0])/sampFreq
finalRobotTrajectory = np.hstack((robot.finalTrajectory[:,0:4],robotTime.reshape((-1,1))))

## ANALYSIS
# Calculate number of collisions with target
threshold = targetSize + robotSize
dist2Target = np.array(targetDist)[:,0]
numCollisions = np.count_nonzero(dist2Target<threshold)
# print(f"{numCollisions} collisions from {threshold} pixels")
print("------------Analysis------------")
print(f"Collisions: {numCollisions}")

# wStar Analytics
dist2WStar = np.array(targetDist)[:,1]
atWStar = np.count_nonzero(dist2WStar<threshold)
# print(f"The robot was at w* {atWStar} times")

if np.count_nonzero(dist2WStar<threshold) == 0:
    print("Never Arrived")
else:
    firstArrival = robotTime[dist2WStar<threshold][0] #IDK WHAT THE TIME UNITS ARE
    # print(f"The robot first arrived at {firstArrival} second(s)")
    print(f"Time of Arrival: {firstArrival}")

if np.count_nonzero(dist2WStar<threshold) == 0:
    print("Never Arrived")
else:
    # Once the robot arrived, what percentage was the robot able to follow the moving target
    indArrival = np.argmax(dist2WStar<threshold)
    indFinal = robotTime.shape[0]-1
    percentageFollowing = (atWStar / (indFinal - indArrival))*100
    # print(f"Once the robot arrived to w*, it was able to follow the target {percentageFollowing:.2f}% of the time")
    print(f"% Followed: {percentageFollowing}")

if np.count_nonzero(dist2WStar<threshold) == 0:
    print("Never Arrived")
else:
    # Average Distance
    avgdist2Target = np.average(dist2Target[indArrival:])
    print(f"Avg Distance to Target: {avgdist2Target}")


anim, fig = create_animation(finalTargetTrajectory[stationaryTime:,:],finalRobotTrajectory[stationaryTime:,:],tf,arena,targetUncertainty = uncertaintyArray[stationaryTime:,:],robotSize=robotSize,targetSize=targetSize,tailLength=20,n_frames=60,uncertaintyFilterFlag=1)
# anim, fig = create_animation(target.trajectory,robot.trajectory[:,0:5],tf,arena,targetUncertainty = targetUncertainty,robotSize=robotSize,targetSize=targetSize,tailLength=20,n_frames=60)


plt.close()
writergif = animation.PillowWriter(fps=15) 
anim.save('./animation.gif', writer=writergif)

anim








------------Analysis------------
Collisions: 0
Time of Arrival: 21.24
% Followed: 96.28387096774193
Avg Distance to Target: 3.7225912823645775


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9d345d04-da85-4b95-91f8-a2840961b3ae' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>